#### Устанавливаем необходимые библиотеки

In [1]:
!pip install transformers
!pip install mendelai-brat-parser
!pip install smart-open

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 68.2 MB/s 
     |████████████████████████████████| 182 kB 55.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mendelai-brat-parser: filename=mendelai_brat_parser-0.0.11-py3-none-any.whl size=4944 sha256=cf6fb426514852fa2efd631094fa6b6c3fe85ae942871f57344760a0d13e161c
  Stored in directory: /root/.cache/pip/wheels/90/a7/ff/138853d8196095fec56e0a97779a96d754b98f169c063beca3
Successfully built mendelai-brat-parser
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import pipeline
from termcolor import colored
import torch

#### Создаем дополнительный класс для удобной работы с моделью

In [3]:
class Ner_Extractor:
    """
    Labeling each token in sentence as named entity

    :param model_checkpoint: name or path to model 
    :type model_checkpoint: string
    """
    
    def __init__(self, model_checkpoint: str):
        self.token_pred_pipeline = pipeline("token-classification", 
                                            model=model_checkpoint, 
                                            aggregation_strategy="average")
    
    @staticmethod
    def text_color(txt, txt_c="blue", txt_hglt="on_yellow"):
        """
        Coloring part of text 
        
        :param txt: part of text from sentence 
        :type txt: string
        :param txt_c: text color  
        :type txt_c: string        
        :param txt_hglt: color of text highlighting  
        :type txt_hglt: string
        :return: string with color labeling
        :rtype: string
        """
        return colored(txt, txt_c, txt_hglt)
    
    @staticmethod
    def concat_entities(ner_result):
        """
        Concatenation entities from model output on grouped entities
        
        :param ner_result: output from model pipeline 
        :type ner_result: list
        :return: list of grouped entities with start - end position in text
        :rtype: list
        """
        entities = []
        prev_entity = None
        prev_end = 0
        for i in range(len(ner_result)):
            
            if (ner_result[i]["entity_group"] == prev_entity) &\
               (ner_result[i]["start"] == prev_end):
                
                entities[i-1][2] = ner_result[i]["end"]
                prev_entity = ner_result[i]["entity_group"]
                prev_end = ner_result[i]["end"]
            else:
                entities.append([ner_result[i]["entity_group"], 
                                 ner_result[i]["start"], 
                                 ner_result[i]["end"]])
                prev_entity = ner_result[i]["entity_group"]
                prev_end = ner_result[i]["end"]
        
        return entities
    
    
    def colored_text(self, text: str, entities: list):
        """
        Highlighting in the text named entities
        
        :param text: sentence or a part of corpus
        :type text: string
        :param entities: concated entities on groups with start - end position in text
        :type entities: list
        :return: Highlighted sentence
        :rtype: string
        """
        colored_text = ""
        init_pos = 0
        for ent in entities:
            if ent[1] > init_pos:
                colored_text += text[init_pos: ent[1]]
                colored_text += self.text_color(text[ent[1]: ent[2]]) + f"({ent[0]})"
                init_pos = ent[2]
            else:
                colored_text += self.text_color(text[ent[1]: ent[2]]) + f"({ent[0]})"
                init_pos = ent[2]
        
        return colored_text
    
    
    def get_entities(self, text: str):
        """
        Extracting entities from text with them position in text
        
        :param text: input sentence for preparing
        :type text: string
        :return: list with entities from text
        :rtype: list
        """
        assert len(text) > 0, text
        entities = self.token_pred_pipeline(text)
        concat_ent = self.concat_entities(entities)
        
        return concat_ent
    
    
    def show_ents_on_text(self, text: str):
        """
        Highlighting named entities in input text 
        
        :param text: input sentence for preparing
        :type text: string
        :return: Highlighting text
        :rtype: string
        """
        assert len(text) > 0, text
        entities = self.get_entities(text)
        
        return self.colored_text(text, entities)

#### Инициализируем модель с предобученными весами

In [4]:
## init model for inference
extractor = Ner_Extractor(model_checkpoint = "surdan/LaBSE_ner_nerel")

Downloading:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

#### Пример текста из тестовой выбороки

In [7]:
seq_example = open("test.txt").read()
print(seq_example)


          отсутствие средств на приобретение лекарств,
          низкая социальная культура,
          малая плотность населения,
          высокая степень алкоголизации населения поселения.
Многие больные обращаются за медицинской помощью лишь в случаях крайней необходимости, при значительной запущенности заболевания и утяжелении самочувствия.
2.10 СОЦИАЛЬНАЯ ЗАЩИТА НАСЕЛЕНИЯ
На территории Полдневицкого сельского поселения осуществляют свою деятельность 1 специалист по соц. защите совместно с 5 социальными работниками.
2.11 ЖИЛИЩНЫЙ ФОНД
Состояние жилищно - коммунальной сферы сельского поселения
Данные о существующем жилищном фонде

Жители Полдневицкого сельского поселения не участвуют в  программах по обеспечению жильем: «Жилье молодым семьям»,  «Молодые специалисты на селе»  по причине отсутствия рабочих мест и отсутствия благоустроенности поселения.
  Услуги ЖКХ не предоставляются по причине отсутствия МКД и управляющей компании. Тепло снабжение - печное отопление, водоснабжен

#### Пример меток нашей модели

In [8]:
## get list of entities from sentence
l_entities = extractor.get_entities(seq_example)
pred = [0] * len(seq_example)
print(l_entities)
for t in l_entities:
    for j in range(t[1], t[2] + 1):
        pred[j] = t[0]

[['ORDINAL', 352, 353], ['LAW', 354, 356], ['LAW', 368, 374], ['DISTRICT', 399, 432], ['NUMBER', 464, 465], ['PROFESSION', 466, 491], ['NUMBER', 504, 505], ['PROFESSION', 506, 529], ['ORDINAL', 531, 532], ['NUMBER', 533, 535], ['DISTRICT', 654, 687], ['DISTRICT', 1120, 1153], ['ORDINAL', 2056, 2057], ['NUMBER', 2243, 2244], ['ORDINAL', 2507, 2508], ['ORDINAL', 2509, 2510], ['ORDINAL', 2711, 2712], ['ORDINAL', 2713, 2714]]


#### Чтение файла меток из тестовой выборки

In [9]:
from brat_parser import get_entities_relations_attributes_groups

entities, relations, attributes, groups = get_entities_relations_attributes_groups("test.ann")

#### Пример меток из тестовой выборки

In [10]:
print(entities)
labels = [0] * len(seq_example)
for val in entities.values():
    for j in range(val.span[0][0], val.span[0][1] + 1):
        labels[j] = val.type

{'T1': Entity(id='T1', type='BIN', span=((12, 22),), text='отсутствие'), 'T2': Entity(id='T2', type='ECO', span=((23, 55),), text='средств на приобретение лекарств'), 'T4': Entity(id='T4', type='QUA', span=((68, 74),), text='низкая'), 'T5': Entity(id='T5', type='SOC', span=((75, 94),), text='социальная культура'), 'T7': Entity(id='T7', type='MET', span=((113, 132),), text='плотность населения'), 'T8': Entity(id='T8', type='QUA', span=((145, 152),), text='высокая'), 'T9': Entity(id='T9', type='MET', span=((153, 194),), text='степень алкоголизации населения поселения'), 'T10': Entity(id='T10', type='SOC', span=((357, 384),), text='СОЦИАЛЬНАЯ ЗАЩИТА НАСЕЛЕНИЯ'), 'T11': Entity(id='T11', type='BIN', span=((433, 445),), text='осуществляют'), 'T14': Entity(id='T14', type='ECO', span=((536, 549),), text='ЖИЛИЩНЫЙ ФОНД'), 'T16': Entity(id='T16', type='ECO', span=((631, 645),), text='жилищном фонде'), 'T17': Entity(id='T17', type='BIN', span=((805, 815),), text='отсутствия'), 'T18': Entity(id='T

#### Специальный словарь для сопоставления меток нашей намодели метками из тестовой выборки

In [14]:
class_map = {
    "AGE": "MET",
    "AWARD": "QUA",
    "CITY": "SOC",
    "COUNTRY": "SOC",
    "CRIME": "ECO",
    "DATE": "MET",
    "DISEASE": "ACT",
    "DISTRICT": "ECO",
    "EVENT": "ACT",
    "FACILITY": "ECO",
    "FAMILY": "INST",
    "IDEOLOGY": "ECO",
    "LAW": "ECO",
    "LOCATION": "MET",
    "MONEY": "MET",
    "NATIONALITY": "SOC",
    "NUMBER": "MET",
    "ORDINAL": "MET",
    "ORGANIZATION": "ECO",
    "PENALTY": "ECO",
    "PERCENT": "MET",
    "PERSON": "ECO",
    "PRODUCT": "ECO",
    "PROFESSION": "INST",
    "RELIGION": "INST",
    "STATE_OR_PROVINCE": "CMP",
    "TIME": "MET",
    "WORK_OF_ART": "ACT",
}

#### Создание метода ждя подсчета recall, precision и f1 и запуск его на всей тестовой выборке

In [32]:
from brat_parser import get_entities_relations_attributes_groups
import numpy
import os

def get_f1(filename, extractor):
    seq_example = open(filename + ".txt").read()
    #calc preds
    l_entities = extractor.get_entities(seq_example)
    pred = [0] * len(seq_example)
    for t in l_entities:
        for j in range(t[1], t[2] + 1):
            pred[j] = t[0]
    #read labels
    entities, relations, attributes, groups = get_entities_relations_attributes_groups(filename + ".ann")
    labels = [0] * len(seq_example)
    for val in entities.values():
        for j in range(val.span[0][0], val.span[0][1] + 1):
            labels[j] = val.type
    #calc f1
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for j in range(len(seq_example)):
      if pred[j] != 0 and labels[j] !=0 and class_map[pred[j]] == labels[j]:
          tp += 1
      elif  pred[j] == labels[j] == 0:
          tn += 1
      elif (labels[j] != 0 and pred[j] == 0) or (labels[j] != 0 and pred[j] != 0 and class_map[pred[j]] != labels[j]):
          fn += 1
      else:
          fp += 1
    recall = tp / (tp + fn + 0.00001)
    precision = tp / (tp + fp + 0.00001)
    f1 = (2 * precision * recall) / (precision + recall + 0.00001)
    return f1

extractor = Ner_Extractor(model_checkpoint = "surdan/LaBSE_ner_nerel")
f1s = list()
folder_name = "test"
for path in os.listdir(folder_name):
    if ".ann" in path:
        continue
    f1s.append(get_f1(folder_name + "/" + path[:-4], extractor))
print(numpy.mean(f1s))

0.238456199576506
